In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from generator import XraySequence
import matplotlib.pyplot as PIL
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
import pandas as pd 
import numpy as np
from skimage.transform import resize
from collections import Counter
import os



In [ ]:
# data from https://github.com/zeeshannisar/COVID-19/blob/master/Implementations/dataset%2002/DenseNet169-Transfer%20Learning-COVID19-dataset%2002.ipynb
dataDir = 'covid_data'
train_x = np.load(dataDir+'/train_images.npy')
train_y = np.load(dataDir+'/train_labels.npy')
test_x = np.load(dataDir+'/test_images.npy')
test_y = np.load(dataDir+'/test_labels.npy')

print('Training Images: {} | Test Images: {}'.format(train_x.shape, test_x.shape))
print('Training Labels: {} | Test Labels: {}'.format(train_y.shape, test_y.shape))

# Data Normalization

print('Train: {} , {} | Test: {} , {}'.format(train_x.min(), train_x.max(), test_x.min(), test_x.max()))

train_x/=255.0
test_x/=255.0

print('Train: {} , {} | Test: {} , {}'.format(train_x.min(), train_x.max(), test_x.min(), test_x.max()))

In [ ]:
# Class Mapping 
print('0:Normal | 1:Pneumonia | 2:COVID-19')

# Distribution of images in each class for Training-set
print(Counter(train_y))

# Distribution of images in each class for Test-set
print(Counter(test_y))

#Make Labels Categorical
train_y_oneHot = tf.one_hot(train_y, depth=3) 
test_y_oneHot = tf.one_hot(test_y, depth=3)

print('Training Labels: {} | Test Labels: {}'.format(train_y_oneHot.shape, test_y_oneHot.shape))

In [ ]:
def get_base_model(weights_path="pre-trained_weight.h5", input_shape=(224, 224, 3)):

    base_weights = None

    img_input = Input(shape=input_shape)

    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights=base_weights,
        pooling="avg")

    x = base_model.output
    predictions = tf.keras.layers.Dense(14, activation='sigmoid', name='predictions')(x)
    model = Model(inputs=img_input, outputs=predictions)

    print(f"load model weights_path: {weights_path}")
    model.load_weights(weights_path)
    
    return model

def DenseNet121_Model():
    baseModel = get_base_model()
    print(baseModel.summary())

    # construct the head of the model that will be placed on top of the the base model
    output = tf.keras.layers.AveragePooling2D(pool_size=(4, 4))(baseModel.layers[-3].output) # remove the FC layers 
    output = tf.keras.layers.Flatten(name="flatten")(output)
    output = tf.keras.layers.Dense(512, activation="relu")(output)
    output = tf.keras.layers.Dropout(0.25)(output)
    output = Dense(3,activation="softmax")(output)

    # place the head FC model on top of baseModel
    model = tf.keras.Model(inputs=baseModel.input, outputs=output)
    # print(model.summary())

    # loop over all layers in the base model and freeze them so they will not be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
    return model

model = DenseNet121_Model()
# model.summary()


In [ ]:
INIT_LR = 0.001
EPOCHS = 50
BATCHSIZE = 32 
optimizer = tf.keras.optimizers.Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()])
print(model.summary())

In [ ]:
trainAug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15, fill_mode="nearest")

modelPath = './saved Models/Densenet121_covid'
if not os.path.exists(modelPath):
  os.makedirs(modelPath)
  print('Model Directory Created')
else:
  print('Model Directory Already Exists')

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(modelPath+'/denseNet121best-model.h5', monitor='val_categorical_accuracy',
                                                      verbose=1, save_best_only=True, mode='auto')

STEP_TRAIN = len(train_x) // BATCHSIZE
STEP_TEST = len(test_x) // BATCHSIZE
modelHistory = model.fit(trainAug.flow(train_x, train_y_oneHot, batch_size=BATCHSIZE), epochs=EPOCHS, verbose=1, callbacks=[model_checkpoint],
                         validation_data=(test_x, test_y_oneHot), shuffle = True, steps_per_epoch=STEP_TRAIN, validation_steps=STEP_TEST)
tf.keras.models.save_model(model, modelPath+'/denseNet121-model.h5', overwrite=True, include_optimizer=True, save_format=None,
                           signatures=None, options=None)